# 🛡️ NIDS Model Evaluation Dashboard

Dashboard interattiva per valutare, confrontare e spiegare i modelli di Network Intrusion Detection.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import glob
import shap
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# CONFIGURAZIONE PATH
# Se il notebook è in 'notebooks/', torniamo indietro di uno per trovare 'data' e 'models'
BASE_DIR = '../'
DATA_DIR = os.path.join(BASE_DIR, 'data/processed/CICIOT23')
MODELS_DIR = os.path.join(BASE_DIR, 'models')

print("📂 Directory configurate:")
print(f"   Data: {DATA_DIR}")
print(f"   Models: {MODELS_DIR}")

## 1. Caricamento Dati (Test Set)

In [ ]:
N_SAMPLES = 50000 # Riduci se la RAM è piena

try:
    print("⏳ Caricamento Test Set...")
    test_path = os.path.join(DATA_DIR, 'test_processed.pkl')
    
    df_test = pd.read_pickle(test_path)
    
    if len(df_test) > N_SAMPLES:
        df_test_sample = df_test.sample(n=N_SAMPLES, random_state=42)
        print(f"⚠️ Uso un sample di {N_SAMPLES} righe (Totale: {len(df_test):,})")
    else:
        df_test_sample = df_test
        
    le_path = os.path.join(DATA_DIR, 'label_encoder.pkl')
    label_encoder = joblib.load(le_path)
    class_names = label_encoder.classes_
    print(f"✅ Classi caricate: {class_names}")

    y_test = df_test_sample['y_macro_encoded']
    X_test = df_test_sample.drop(columns=['y_macro_encoded', 'y_specific'], errors='ignore')
    feature_names = X_test.columns.tolist()
    
except Exception as e:
    print(f"❌ Errore caricamento dati: {e}")

## 2. Caricamento Modelli Addestrati

In [ ]:
models = {}
# Cerca tutti i file .pkl nelle sottocartelle di models/
model_files = glob.glob(os.path.join(MODELS_DIR, '*/*.pkl'))

print(f"🔍 Trovati {len(model_files)} modelli:")
for p in model_files:
    model_name = os.path.basename(p).replace('.pkl', '')
    algo = os.path.basename(os.path.dirname(p))
    display_name = f"{algo} ({model_name})"
    print(f"  - Carico: {display_name}...")
    try:
        models[display_name] = joblib.load(p)
    except Exception as e:
        print(f"    ❌ Errore: {e}")

## 3. Confronto Metriche

In [ ]:
results = []
for name, model in models.items():
    print(f"⚡ Test {name}...")
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    results.append({'Model': name, 'Accuracy': acc, 'F1-Score': f1})

if results:
    df_res = pd.DataFrame(results).sort_values('F1-Score', ascending=False)
    plt.figure(figsize=(10, 5))
    sns.barplot(data=df_res, x='F1-Score', y='Model', palette='viridis')
    plt.title('Classifica Modelli (F1-Score)')
    plt.xlim(0.8, 1.0)
    plt.show()
    display(df_res)

## 4. Matrici di Confusione

In [ ]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix: {name}')
    plt.ylabel('Reale')
    plt.xlabel('Predetto')
    plt.show()

## 5. Explainable AI (SHAP)

In [ ]:
if results:
    best_name = df_res.iloc[0]['Model']
    best_model = models[best_name]
    print(f"🔬 Analisi SHAP su: {best_name}")
    
    try:
        # Usa un subset piccolissimo per SHAP (è lento)
        X_shap = X_test.iloc[:100]
        explainer = shap.TreeExplainer(best_model)
        shap_values = explainer.shap_values(X_shap)
        
        # Per multiclasse, shap_values è una lista. Prendiamo la classe DDoS (es. indice 2)
        target_idx = 2 
        if len(shap_values) > target_idx:
            print(f"📊 Spiegazione per classe: {class_names[target_idx]}")
            shap.summary_plot(shap_values[target_idx], X_shap)
        else:
            shap.summary_plot(shap_values, X_shap)
            
    except Exception as e:
        print(f"⚠️ SHAP non disponibile: {e}")